# Elixir: try, catch, rescue

### Errors
* Errors (or exceptions) are used when exceptional things happen in the code. A sample error can be retrieved by trying to add a number into an atom.
* A runtime error can be raised any time by using raise/1.
* Other errors can be raised with raise/2 passing the error name and a list of keyword arguments.
* You can define custom errors by creating a module and using the defexception construct inside it; this way, you’ll create an error with the same name as the module it’s defined in. The most common case is to define a custom exception with a message field.
* Errors can be rescued with the try/rescue construct.

In [1]:
:foo + 1

ArithmeticError: 1

In [1]:
raise "oops"

RuntimeError: 1

In [1]:
raise ArgumentError, message: "invalid argument foo"

ArgumentError: 1

In [1]:
defmodule MyError do
  defexception message: "default message"
  end
  

{:module, MyError, <<70, 79, 82, 49, 0, 0, 11, 228, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 1, 72, 0, 0, 0, 33, 14, 69, 108, 105, 120, 105, 114, 46, 77, 121, 69, 114, 114, 111, 114, 8, 95, 95, 105, 110, 102, 111, 95, ...>>, :ok}

In [2]:
raise MyError

MyError: 1

In [2]:
raise MyError, message: "doh!"

MyError: 1

In [2]:
try do
  raise "doh!"
  rescue
    e in RuntimeError -> e
    end

%RuntimeError{message: "doh!"}

In [3]:
try do
  raise "doh!"
  rescue
  RuntimeError -> "Error!"
  end

"Error!"

* In practice, Elixir developers rarely use try/rescue. For example, many languages force you to rescue an error when a file cannot be opened successfully. 
* Elixir instead provides a File.read/1 function which returns a tuple containing information about whether the file was opened successfully.
* When you expect a file to exist (and the lack of that file is truly an error) you may use File.read!/1:

In [4]:
File.read "hello"

{:ok, ""}

In [5]:
File.write "hello", "world"

:ok

In [6]:
File.read "hello"

{:ok, "world"}

In [7]:
File.read! "unknown"

File.Error: 1

* Many std functions follow the pattern of having a counterpart that raises an exception instead of returning tuples to match against. 
* The convention is a function (foo) which returns {:ok, result} or {:error, reason} tuples and another function (foo!) that takes the same arguments and raises an exception if there’s an error.
* In Elixir, we avoid using try/rescue because we don’t use errors for control flow. We take errors literally: they are reserved for unexpected and/or exceptional situations. In case you actually need flow control constructs, throws should be used.

### Throws
* Elixir values can be thrown and later be caught. throw and catch are reserved for situations where it is not possible to retrieve a value unless by using throw and catch.

* Those situations are quite uncommon except when interfacing with libraries that do not provide a proper API. For example, let’s imagine the Enum module did not provide any API for finding a value and that we needed to find the first multiple of 13 in a list of numbers.

In [7]:
try do
  Enum.each -50..50, fn(x) ->
    if rem(x,13) == 0, do: throw(x)
    end
    "got nothing"
    catch
      x -> "got #{x}"
      end
    

"got -39"

* Since Enum does provide a proper API, in practice Enum.find/2 is the way to go.

In [8]:
Enum.find -50..50, &(rem(&1, 13) == 0)

-39

### Exits
* All Elixir code runs inside processes that communicate with each other. When a process dies of “natural causes” (e.g., unhandled exceptions), it sends an exit signal. A process can also die by explicitly sending an exit signal.

![exit](pix/exit.png)

* Exit can also be caught using try/catch.

In [9]:
try do
  exit "i am exiting"
catch
  :exit, _ -> "not really"
  end
  

"not really"

* Exit signals are an important part of the fault tolerant system provided by the Erlang VM. Processes usually run under supervision trees which are themselves processes that listen to exit signals from the supervised processes. Once an exit signal is received, the supervision strategy kicks in and the supervised process is restarted.

* It is exactly this supervision system that makes constructs like try/catch and try/rescue so uncommon in Elixir. Instead of rescuing an error, we’d rather “fail fast” since the supervision tree will guarantee our application will go back to a known initial state after the error.

### After
* Sometimes it’s necessary to ensure that a resource is cleaned up after some action that could potentially raise an error. The try/after construct allows you to do that. For example, we can open a file and use an after clause to close it–even if something goes wrong.

In [10]:
{:ok, file} = File.open "sample", [:utf8, :write]

{:ok, #PID<0.270.0>}

In [11]:
try do
  IO.write file, "wazoo"
  raise "oops, something went wrong"
  after
    File.close(file)
    end

RuntimeError: 1

* The after clause will be executed regardless of whether or not the tried block succeeds. * Note: if a linked process exits, this process will exit and the after clause will not run - thus after provides only a soft guarantee. 
* Luckily, files in Elixir are also linked to the current processes - they will always get closed if the current process crashes, independent of the after clause. You will find the same to be true for other resources like ETS tables, sockets, ports and more.

* Sometimes you may want to wrap the entire body of a function in a try construct, often to guarantee some code will be executed afterwards. In such cases, Elixir allows you to omit the try line.

In [11]:
# ****** BUG ****** TODO ******

defmodule RunAfter do
  def without_even_trying do
    raise "oops"
  after
    IO.puts "cleaning up"
  end
end    

{:module, RunAfter, <<70, 79, 82, 49, 0, 0, 5, 96, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 230, 0, 0, 0, 19, 15, 69, 108, 105, 120, 105, 114, 46, 82, 117, 110, 65, 102, 116, 101, 114, 8, 95, 95, 105, 110, 102, 111, ...>>, {:without_even_trying, 0}}

In [12]:
RunAfter.without_even_trying

RuntimeError: 1

### Else
* If an else block is present, it will match on the results of the try block whenever the try block finishes without a throw or error.
* Exceptions in the else block are not caught. If no pattern inside the else block matches, an exception will be raised; this exception is not caught by the current try/catch/rescue/after block.

In [12]:
x = 2

2

In [13]:
try do
 1/x
 rescue
   ArithmeticError -> :infinity
   else
     y when y<1 and y>-1 -> :small
     _ -> :large
     end

:small

### Variables scope
* Variables defined inside try/catch/rescue/after blocks do not leak to the outer context. This is because the try block may fail and as such the variables may never be bound in the first place. In other words, this code is invalid.

In [14]:
try do
  raise "fail"
  what_happened = :did_not_raise
  rescue
  _ -> what_happened = :rescued
  end

  nofile:3

  nofile:5



:rescued

In [15]:
what_happened = 
  try do
    raise "fail"
    :did_not_raise
    rescue
    _ -> :rescued
    end

:rescued